In [1]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

In [2]:
# System imports
import os
import sys
import yaml

# External imports
import scipy as sp
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

sys.path.append('..')
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from LightningModules.GNN.Models.ecgnn import ECGNN

In [4]:
with open("example_gnn.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

In [ ]:
model = ECGNN(hparams)
logger = TensorBoardLogger("tb_ecgnn_logs",name = "ecgnn")
trainer = Trainer(max_epochs=20,logger = logger)
trainer.fit(model)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name          | Type        | Params
----------------------------------------------
0 | input_network | Sequential  | 96    
1 | node_network  | NodeNetwork | 1.2 K 
2 | edge_network  | EdgeNetwork | 1.6 K 
----------------------------------------------
3.0 K     Trainable params
0         Non-trainable params
3.0 K     Total params
0.012     Total estimated model params size (MB)


Epoch 0:   2%|█▎                                                | 2/80 [06:27<4:11:37, 193.56s/it, loss=0.912, v_num=8]

In [ ]:
test_results = trainer.test(ckpt_path=None)
print(test_results)

In [ ]:
os.environ['TENSORBOARD_BINARY'] = "/global/homes/j/jferguso/.conda/envs/exatrkx-tracking/bin/tensorboard"
%tensorboard --logdir tb_ecgnn_logs/ --port 0
nersc_tensorboard_helper.tb_address()
